In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
"""    # Drop normalized columns
    columns = []
    for i in df.columns:
        columns.append(i)
    columns = str(columns)
    normalized_columns = re.findall(r'(smart_\d+_normalized)', columns)
    df.drop(columns=normalized_columns, inplace=True)"""

"    # Drop normalized columns\n    columns = []\n    for i in df.columns:\n        columns.append(i)\n    columns = str(columns)\n    normalized_columns = re.findall(r'(smart_\\d+_normalized)', columns)\n    df.drop(columns=normalized_columns, inplace=True)"

In [3]:
df = pd.read_csv('hard_drives_smart_5.csv')

In [4]:
df.shape

(169073, 10)

In [5]:
df.head(10)

,Unnamed: 0,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_197_raw),max(smart_198_raw)
0,0,176FT026T,TOSHIBA MQ01ABF050M,500107862016,0,9965.0,0.0,NaN,0.0,0.0
1,1,17eddeea3c620010,DELLBOSS VD,480036847616,0,NaN,NaN,NaN,NaN,NaN
2,2,18K0A02ZF97G,TOSHIBA MG07ACA14TA,14000519643136,1,99.0,43.0,NaN,0.0,0.0
3,3,2AGHLPBY,HGST HUH721212ALN604,12000138625024,0,990.0,0.0,NaN,0.0,0.0
4,4,2AGM5WEY,HGST HUH721212ALN604,12000138625024,0,978.0,0.0,NaN,0.0,0.0
5,5,2AGME31Y,HGST HUH721212ALN604,12000138625024,0,984.0,0.0,NaN,0.0,0.0
6,6,2AGMGLBY,HGST HUH721212ALN604,-1,0,NaN,NaN,NaN,NaN,NaN
7,7,2AGN05VY,HGST HUH721212ALN604,12000138625024,0,982.0,0.0,NaN,0.0,0.0
8,8,2AGN810Y,HGST HUH721212ALN604,-1,0,NaN,NaN,NaN,NaN,NaN
9,9,2AGNEK8Y,HGST HUH721212ALN604,-1,0,NaN,NaN,NaN,NaN,NaN


In [6]:
df['max(failure)'].value_counts()

0    163141
1      5932
Name: max(failure), dtype: int64

In [7]:
5932/(5932+163141) * 100

3.50854364682711

In [8]:
def prepare(df):
    
    # Convert capacity column from bytes to gigabytes
    df['capacity_bytes'] = round((df['capacity_bytes']/ 1_000_000_000),1)

    # Convert power hours to years
    df['max(smart_9_raw)'] = round((df['max(smart_9_raw)']/ 8760),1)

    # Drop unnamed column
    df.drop(columns=('Unnamed: 0'), inplace = True)
    
    # Rename columns appropriately
    df.rename(columns={'capacity_bytes':'capacity_gigabytes','max(failure)':'failure','max(smart_9_raw)':'drive_age_in_years','max(smart_5_raw)':'reallocated_sectors_count','max(smart_187_raw)':'reported_uncorrectable_errors','max(smart_197_raw)':'current_pending_sector_count','max(smart_198_raw)':'uncorrectable_sector_count'}, inplace = True)
    
    return df

In [9]:
# 'max(smart_188_raw)':'command_timeout',

In [10]:
prepare(df)

,serial_number,model,capacity_gigabytes,failure,drive_age_in_years,reallocated_sectors_count,reported_uncorrectable_errors,current_pending_sector_count,uncorrectable_sector_count
0,176FT026T,TOSHIBA MQ01ABF050M,500.1,0,1.1,0.0,NaN,0.0,0.0
1,17eddeea3c620010,DELLBOSS VD,480.0,0,NaN,NaN,NaN,NaN,NaN
2,18K0A02ZF97G,TOSHIBA MG07ACA14TA,14000.5,1,0.0,43.0,NaN,0.0,0.0
3,2AGHLPBY,HGST HUH721212ALN604,12000.1,0,0.1,0.0,NaN,0.0,0.0
4,2AGM5WEY,HGST HUH721212ALN604,12000.1,0,0.1,0.0,NaN,0.0,0.0
5,2AGME31Y,HGST HUH721212ALN604,12000.1,0,0.1,0.0,NaN,0.0,0.0
6,2AGMGLBY,HGST HUH721212ALN604,-0.0,0,NaN,NaN,NaN,NaN,NaN
7,2AGN05VY,HGST HUH721212ALN604,12000.1,0,0.1,0.0,NaN,0.0,0.0
8,2AGN810Y,HGST HUH721212ALN604,-0.0,0,NaN,NaN,NaN,NaN,NaN
9,2AGNEK8Y,HGST HUH721212ALN604,-0.0,0,NaN,NaN,NaN,NaN,NaN


In [12]:
df.dtypes

serial_number                     object
model                             object
capacity_gigabytes               float64
failure                            int64
drive_age_in_years               float64
reallocated_sectors_count        float64
reported_uncorrectable_errors    float64
current_pending_sector_count     float64
uncorrectable_sector_count       float64
dtype: object

In [14]:
last = pd.read_csv('2019-09-30.csv')

In [18]:
last['smart_9_raw'][last.serial_number == '176FT026T']

97652    18173.0
Name: smart_9_raw, dtype: float64

In [19]:
df.model.nunique()

103

In [20]:
df.serial_number.nunique()

162030

document questions before exploration
spend time as a group to come up with questions useful for final goal
curiosities, hunches, etc. 
things that can be useful

work with most consistent chunck of data, dont need to go back further than 2016